In [1]:
pip install pandas matplotlib seaborn


Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys

# Configuração do tema de gráficos
sns.set_theme()

# Função para plotar a tabela dinâmica
def plot_pivot_table(df: pd.DataFrame, value: str, index: str, func: str, ylabel: str, xlabel: str, opcao: str='nenhuma') -> None:
    if opcao == 'nenhuma':
        pd.pivot_table(data=df, values=value, index=index, aggfunc=func).plot(figsize=[15, 5])
    elif opcao == 'sort':
        pd.pivot_table(data=df, values=value, index=index, aggfunc=func).sort_values(value).plot(figsize=[15, 5])
    elif opcao == 'unstack':
        pd.pivot_table(data=df, values=value, index=index, aggfunc=func).unstack().plot(figsize=[15, 5])
    plt.ylabel(ylabel)
    plt.xlabel(xlabel)
    return None

# Função principal para processar os meses fornecidos como argumento
def gerar_analise(meses):
    for mes in meses:
        try:
            # Lê os dados do CSV correspondente ao mês
            sinasc = pd.read_csv(f'SINASC_RO_2019_{mes}.csv')
            
            # Obtém a data mínima e máxima
            data_inicial = sinasc.DTNASC.min()
            data_final = sinasc.DTNASC.max()

            # Obtém a data máxima no formato YYYY-MM
            max_data = sinasc.DTNASC.max()[:7]

            # Cria diretório para salvar os gráficos
            os.makedirs(f'./output/figs/{max_data}', exist_ok=True)

            # Gera gráficos
            plot_pivot_table(df=sinasc, value='IDADEMAE', index='DTNASC', func='count', ylabel='Quantidade de nascimentos', xlabel='Data de nascimento')
            plt.savefig(f'./output/figs/{max_data}/Quantidade de nascimentos.png')
            plt.close()

            plot_pivot_table(df=sinasc, value='IDADEMAE', index=['DTNASC', 'SEXO'], func='mean', ylabel='Média da idade das mães', xlabel='Data de nascimento', opcao='unstack')
            plt.savefig(f'./output/figs/{max_data}/Média da idade das mães por sexo.png')
            plt.close()

            plot_pivot_table(df=sinasc, value='PESO', index=['DTNASC', 'SEXO'], func='mean', ylabel='Média do peso dos bebês', xlabel='Data de nascimento', opcao='unstack')
            plt.savefig(f'./output/figs/{max_data}/Média do peso dos bebês por sexo.png')
            plt.close()

            plot_pivot_table(df=sinasc, value='APGAR1', index='ESCMAE', func='median', ylabel='Mediana do APGAR1', xlabel='Escolaridade', opcao='sort')
            plt.savefig(f'./output/figs/{max_data}/Mediana do APGAR1 por escolaridade das mães.png')
            plt.close()

            plot_pivot_table(df=sinasc, value='APGAR1', index='GESTACAO', func='mean', ylabel='Média do APGAR1', xlabel='Gestação', opcao='sort')
            plt.savefig(f'./output/figs/{max_data}/Média do APGAR1 por gestação.png')
            plt.close()

            plot_pivot_table(df=sinasc, value='APGAR5', index='GESTACAO', func='mean', ylabel='Média do APGAR5', xlabel='Gestação', opcao='sort')
            plt.savefig(f'./output/figs/{max_data}/Média do APGAR5 por gestação.png')
            plt.close()

            # Printando as informações de data inicial, final e o nome da pasta
            print(f'Data inicial: {data_inicial} Data final: {data_final}')
            print(f'Nome da pasta: {max_data} \n')

        except FileNotFoundError:
            print(f'Arquivo SINASC_RO_2019_{mes}.csv não encontrado! Verifique o nome do arquivo e o diretório.')
        except Exception as e:
            print(f'Ocorreu um erro ao processar o mês {mes}: {e}')

if __name__ == "__main__":
    # Verifica se foram passados argumentos suficientes
    if len(sys.argv) < 2:
        print("Uso: python gerar_analise.py MAR ABR MAI JUN JUL")
    else:
        # Pega a lista de meses, filtrando argumentos que não são meses
        meses = [arg for arg in sys.argv[1:] if arg.isalpha()]
        gerar_analise(meses)

